## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# File to Load
city_data_to_load = os.path.join("..", "Weather_Database", "WeatherPy_Database.csv")

# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv(city_data_to_load)
city_data_df.head()


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Punta Arenas,CL,-53.1500,-70.9167,48.31,71,75,8.05,broken clouds
1,1,Comodoro Rivadavia,AR,-45.8667,-67.5000,84.06,21,0,14.97,clear sky
2,2,Leshukonskoye,RU,64.8989,45.7622,8.20,98,97,2.44,overcast clouds
3,3,Te Anau,NZ,-45.4167,167.7167,62.02,79,100,4.99,overcast clouds
4,4,Saint-Augustin,CA,51.2260,-58.6502,23.72,89,100,23.73,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 100


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Comodoro Rivadavia,AR,-45.8667,-67.5000,84.06,21,0,14.97,clear sky
7,7,Vaini,TO,-21.2000,-175.2000,86.16,74,40,5.75,scattered clouds
9,9,Kapaa,US,22.0752,-159.3190,80.58,67,0,4.00,clear sky
15,15,Rikitea,PF,-23.1203,-134.9692,78.66,72,100,7.27,overcast clouds
19,19,Faanui,PF,-16.4833,-151.7500,81.14,72,17,19.46,few clouds
20,20,Mahebourg,MU,-20.4081,57.7000,80.26,94,75,31.07,moderate rain
22,22,Bambous Virieux,MU,-20.3428,57.7575,80.22,94,75,31.07,moderate rain
25,25,Daru,PG,-9.0763,143.2092,79.41,71,100,6.04,overcast clouds
27,27,Arraial Do Cabo,BR,-22.9661,-42.0278,76.96,94,75,3.44,broken clouds
33,33,Makakilo City,US,21.3469,-158.0858,80.69,59,0,8.05,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                171
City                   171
Country                171
Lat                    171
Lng                    171
Max Temp               171
Humidity               171
Cloudiness             171
Wind Speed             171
Current Description    171
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# Not necessary

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Comodoro Rivadavia,AR,84.06,clear sky,-45.8667,-67.5000,
7,Vaini,TO,86.16,scattered clouds,-21.2000,-175.2000,
9,Kapaa,US,80.58,clear sky,22.0752,-159.3190,
15,Rikitea,PF,78.66,overcast clouds,-23.1203,-134.9692,
19,Faanui,PF,81.14,few clouds,-16.4833,-151.7500,
20,Mahebourg,MU,80.26,moderate rain,-20.4081,57.7000,
22,Bambous Virieux,MU,80.22,moderate rain,-20.3428,57.7575,
25,Daru,PG,79.41,overcast clouds,-9.0763,143.2092,
27,Arraial Do Cabo,BR,76.96,broken clouds,-22.9661,-42.0278,
33,Makakilo City,US,80.69,clear sky,21.3469,-158.0858,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except:
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.count()

City                   171
Country                171
Max Temp               171
Current Description    171
Lat                    171
Lng                    171
Hotel Name             171
dtype: int64

In [10]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))